In [6]:
# import math
import numpy as np
import scipy.spatial.distance
import scipy.signal
from skimage.segmentation import slic
from skimage.util import img_as_float
from scipy.optimize import minimize

import os
import scipy.io
import cv2
import pywt
import numpy as np

from skimage import io
import itertools

In [12]:
File_path = '/home/nitk/Desktop/Raj_M.Tech_Project/Approach5/Input_Image_Dataset/Location_Appended_Input/HS_images_10'
HS_files = sorted(os.listdir(File_path))
print("HS_files", HS_files)
# num = 0;
save_path = '/home/nitk/Desktop/Raj_M.Tech_Project/Approach5/Output_Image_Dataset/Location_Appended_Images/contrast_featureMap'

HS_files ['0001.mat', '0002.mat', '0006.mat', '0008.mat', '0009.mat', '0010.mat', '0012.mat', '0014.mat', '0015.mat', '0016.mat', '0017.mat', '0018.mat', '0019.mat', '0020.mat', '0021.mat', '0022.mat', '0024.mat', '0026.mat', '0027.mat', '0028.mat', '0029.mat', '0031.mat', '0032.mat', '0033.mat', '0034.mat', '0036.mat', '0037.mat', '0038.mat', '0039.mat', '0040.mat', '0041.mat', '0042.mat', '0043.mat', '0044.mat', '0045.mat', '0046.mat', '0047.mat', '0050.mat', '0051.mat', '0052.mat', '0053.mat', '0054.mat', '0056.mat', '0058.mat', '0061.mat', '0062.mat', '0063.mat', '0064.mat', '0065.mat', '0068.mat', '0069.mat', '0070.mat', '0071.mat', '0072.mat', '0074.mat', '0076.mat', '0077.mat', '0078.mat', '0079.mat', '0082.mat']


In [13]:
def mkdir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [14]:
im_contrast_path = os.path.join(save_path, 'contrast_prior_results')
mkdir(im_contrast_path)

In [22]:
def wavelet_contrast_prior(img):
    # Convert the image to grayscale if it's not already
    if img.ndim == 3:
        img = np.mean(img, axis=2)
    
    # Perform 2D wavelet decomposition using the Haar wavelet
    coeffs = pywt.dwt2(img, 'haar')
    cA, (cH, cV, cD) = coeffs

    # Compute the horizontal and vertical absolute differences of the coefficients
    diff_h = np.abs(cH)
    diff_v = np.abs(cV)

    # Compute the local mean and standard deviation of the coefficient differences
    mu_h = np.mean(diff_h)
    mu_v = np.mean(diff_v)
    std_h = np.std(diff_h)
    std_v = np.std(diff_v)

    # Compute the wavelet-based contrast prior map
    cp_h = (diff_h - mu_h) / (std_h + 1e-8)
    cp_v = (diff_v - mu_v) / (std_v + 1e-8)
    cp = np.sqrt(cp_h ** 2 + cp_v ** 2)

    # Upsample the contrast prior map to the original image size
    cp = cv2.resize(cp, (img.shape[1], img.shape[0]))

    # Normalize the contrast prior map to the range [0, 1]
    cp = (cp - np.min(cp)) / (np.max(cp) - np.min(cp))

    return cp


In [23]:
num = 0
for file in HS_files:
    input_img = os.path.join(File_path, file)
    print("Reading File[%d/%d]: %s" % (num, len(HS_files), input_img))
    print("file name", file)
    mat_dict = scipy.io.loadmat(input_img)
    for key in mat_dict:
        if type(mat_dict[key]) is np.ndarray:
            image_data = mat_dict[key]  # type: np.ndarray
            print("image data shape is", image_data.shape)

    file = file.rsplit('.', 1)
    contrast_featureMap = wavelet_contrast_prior(image_data)

    max1 = np.mean(contrast_featureMap)
    contrast_featureMap = contrast_featureMap//max1
    contrast_featureMap = contrast_featureMap * 255

    contrast_featureMap = cv2.transpose(contrast_featureMap)
    
    # print('contrast_featureMap shape', contrast_featureMap.shape)

    num = num + 1

    save_im_w_featureMap = im_contrast_path + '/' + file[0] + '.png'
    cv2.imwrite(save_im_w_featureMap, contrast_featureMap)

Reading File[0/60]: /home/nitk/Desktop/Raj_M.Tech_Project/Approach5/Input_Image_Dataset/Location_Appended_Input/HS_images_10/0001.mat
file name 0001.mat
image data shape is (1024, 768, 11)
Reading File[1/60]: /home/nitk/Desktop/Raj_M.Tech_Project/Approach5/Input_Image_Dataset/Location_Appended_Input/HS_images_10/0002.mat
file name 0002.mat
image data shape is (1024, 768, 11)
Reading File[2/60]: /home/nitk/Desktop/Raj_M.Tech_Project/Approach5/Input_Image_Dataset/Location_Appended_Input/HS_images_10/0006.mat
file name 0006.mat
image data shape is (1024, 768, 11)
Reading File[3/60]: /home/nitk/Desktop/Raj_M.Tech_Project/Approach5/Input_Image_Dataset/Location_Appended_Input/HS_images_10/0008.mat
file name 0008.mat
image data shape is (1024, 768, 11)
Reading File[4/60]: /home/nitk/Desktop/Raj_M.Tech_Project/Approach5/Input_Image_Dataset/Location_Appended_Input/HS_images_10/0009.mat
file name 0009.mat
image data shape is (1024, 768, 11)
Reading File[5/60]: /home/nitk/Desktop/Raj_M.Tech_Proje